In [1]:

from pathlib import Path
import sys
import pandas as pd

BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
PROC_DIR = BASE_DIR / 'data' / 'processed'
SRC_DIR = BASE_DIR / 'src'

# make src importable
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

from src.cleaning import fill_missing_median, drop_missing, normalize_data

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)

RAW_DIR, PROC_DIR, SRC_DIR


(WindowsPath('C:/Users/24614/bootcamp_Mingxuan_Jiang/homework/homework6/data/raw'),
 WindowsPath('C:/Users/24614/bootcamp_Mingxuan_Jiang/homework/homework6/data/processed'),
 WindowsPath('C:/Users/24614/bootcamp_Mingxuan_Jiang/homework/homework6/src'))

In [2]:

import glob

csvs = sorted(glob.glob(str(RAW_DIR / "*.csv")))
if not csvs:
    # fallback: copy the sample that ships with the repo if for some reason missing
    sample = RAW_DIR / "sample_raw.csv"
    if not sample.exists():
        # minimal reproducibility: create a sample
        import pandas as pd
        pd.DataFrame({
            "age": [23, 35, None, 52, 41, None],
            "income": [42000, 73000, 58000, None, 91000, 73000],
            "score": [0.12, None, 0.88, 0.43, 0.57, None],
            "city": ["NY", "SF", "NY", "LA", None, "LA"],
            "mostly_missing": [None, None, None, None, 1.0, None],
        }).to_csv(sample, index=False)
    csvs = [str(sample)]

raw_path = csvs[0]
df = pd.read_csv(raw_path)
df_orig = df.copy()
raw_path, df.shape


('C:\\Users\\24614\\bootcamp_Mingxuan_Jiang\\homework\\homework6\\data\\raw\\starter_data (1).csv',
 (10, 3))

In [3]:

display(df.head())
display(df.describe(include='all'))
missing_ratio_cols = df.isna().mean().sort_values(ascending=False)
missing_ratio_cols.to_frame('missing_ratio')


,category,value,date
0,A,10,2025-08-01
1,B,15,2025-08-02
2,A,12,2025-08-03
3,B,18,2025-08-04
4,C,25,2025-08-05


,category,value,date
count,10,10.000000,10
unique,3,NaN,10
top,A,NaN,2025-08-01
freq,4,NaN,1
mean,NaN,17.600000,NaN
std,NaN,7.381659,NaN
min,NaN,10.000000,NaN
25%,NaN,12.250000,NaN
50%,NaN,14.500000,NaN
75%,NaN,23.250000,NaN


,missing_ratio
category,0.0
value,0.0
date,0.0


In [4]:

# 1) drop columns with very high missingness
df1 = drop_missing(df, threshold=0.6, axis="columns")

# 2) fill numeric columns with median
df2 = fill_missing_median(df1, columns=None)

# 3) normalize numeric columns
df_clean = normalize_data(df2, columns=None)

df_clean.shape


(10, 3)

In [5]:

out_path = PROC_DIR / "cleaned_data.csv"
df_clean.to_csv(out_path, index=False)
str(out_path), out_path.exists()


('C:\\Users\\24614\\bootcamp_Mingxuan_Jiang\\homework\\homework6\\data\\processed\\cleaned_data.csv',
 True)

In [6]:

summary = {
    "original_shape": df_orig.shape,
    "cleaned_shape": df_clean.shape,
    "original_missing_ratio_mean": float(df_orig.isna().mean().mean()),
    "cleaned_missing_ratio_mean": float(df_clean.isna().mean().mean())
}
summary


{'original_shape': (10, 3),
 'cleaned_shape': (10, 3),
 'original_missing_ratio_mean': 0.0,
 'cleaned_missing_ratio_mean': 0.0}

In [7]:

display(df_orig.describe(include='all'))
display(df_clean.describe(include='all'))


,category,value,date
count,10,10.000000,10
unique,3,NaN,10
top,A,NaN,2025-08-01
freq,4,NaN,1
mean,NaN,17.600000,NaN
std,NaN,7.381659,NaN
min,NaN,10.000000,NaN
25%,NaN,12.250000,NaN
50%,NaN,14.500000,NaN
75%,NaN,23.250000,NaN


,category,value,date
count,10,10.000000,10
unique,3,NaN,10
top,A,NaN,2025-08-01
freq,4,NaN,1
mean,NaN,0.380000,NaN
std,NaN,0.369083,NaN
min,NaN,0.000000,NaN
25%,NaN,0.112500,NaN
50%,NaN,0.225000,NaN
75%,NaN,0.662500,NaN
